# CS246 - Colab 9
## Studying COVID-19

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 204.7MB 65kB/s 
     |████████████████████████████████| 204kB 45.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044184 sha256=e70f0a7aaa5a9f3299abda88f616368bdb9cf187f283fa721aa5b426428e2330
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 35.8 MB of archives.
After this 

Now we authenticate a Google Drive client to download the files we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1qgUxXG2rVDPuTvvoamqDQVNh0DDZ4LUh'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_19-covid-Confirmed.csv')

id='1KMR3I0Mz6XHtv5tsjGIIgyLNS0jmniaA'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_19-covid-Deaths.csv')

id='1wKgm-A6p6K79hmtDJKRKuK-Cf1kzsIhw'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_19-covid-Recovered.csv')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import * 
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.sql.functions as f

from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [6]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [7]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-08-29 05:31:56--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.5.250.138, 52.206.116.16, 18.214.118.253, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.5.250.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  73.5MB/s    in 0.2s    

2020-08-29 05:31:56 (73.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


### Data Loading

In this Colab, we will be analyzing the timeseries data of the Coronavirus COVID-19 Global Cases, collected by Johns Hopkins CSSE.

Here you can check a realtime dashboard based on this dataset: [https://www.arcgis.com/apps/opsdashboard/index.html?fbclid=IwAR2hQKsEZ3D38wVtXGryUhP9CG0Z6MYbUM_boPEaV8FBe71wUvDPc65ZG78#/bda7594740fd40299423467b48e9ecf6](https://www.arcgis.com/apps/opsdashboard/index.html?fbclid=IwAR2hQKsEZ3D38wVtXGryUhP9CG0Z6MYbUM_boPEaV8FBe71wUvDPc65ZG78#/bda7594740fd40299423467b48e9ecf6)

---



*   ```confirmed```: dataframe containing the total number of confirmed COVID-19 cases per day, divided by geographical area
*   ```deaths```: dataframe containing the number of deaths per day due to COVID-19, divided by geographical area
*   ```recovered```: dataframe containing the number of recoevered patients per day, divided by geographical area







In [8]:
confirmed = spark.read.csv('time_series_19-covid-Confirmed.csv', header=True)
deaths = spark.read.csv('time_series_19-covid-Deaths.csv', header=True)
recovered = spark.read.csv('time_series_19-covid-Recovered.csv', header=True)

In [10]:

deaths.take(5)

[Row(Province/State='Anhui', Country/Region='Mainland China', Lat='31.8257', Long='117.2264', 1/22/20='0', 1/23/20='0', 1/24/20='0', 1/25/20='0', 1/26/20='0', 1/27/20='0', 1/28/20='0', 1/29/20='0', 1/30/20='0', 1/31/20='0', 2/1/20='0', 2/2/20='0', 2/3/20='0', 2/4/20='0', 2/5/20='0', 2/6/20='0', 2/7/20='0', 2/8/20='0', 2/9/20='1', 2/10/20='3', 2/11/20='4', 2/12/20='4', 2/13/20='5', 2/14/20='6', 2/15/20='6', 2/16/20='6', 2/17/20='6', 2/18/20='6', 2/19/20='6', 2/20/20='6', 2/21/20='6', 2/22/20='6', 2/23/20='6', 2/24/20='6', 2/25/20='6', 2/26/20='6', 2/27/20='6', 2/28/20='6', 2/29/20='6', 3/1/20='6', 3/2/20='6', 3/3/20='6', 3/4/20='6', 3/5/20='6'),
 Row(Province/State='Beijing', Country/Region='Mainland China', Lat='40.1824', Long='116.4142', 1/22/20='0', 1/23/20='0', 1/24/20='0', 1/25/20='0', 1/26/20='0', 1/27/20='1', 1/28/20='1', 1/29/20='1', 1/30/20='1', 1/31/20='1', 2/1/20='1', 2/2/20='1', 2/3/20='1', 2/4/20='1', 2/5/20='1', 2/6/20='1', 2/7/20='1', 2/8/20='2', 2/9/20='2', 2/10/20='2', 

In [9]:
confirmed.printSchema()

root
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: string (nullable = true)
 |-- Long: string (nullable = true)
 |-- 1/22/20: string (nullable = true)
 |-- 1/23/20: string (nullable = true)
 |-- 1/24/20: string (nullable = true)
 |-- 1/25/20: string (nullable = true)
 |-- 1/26/20: string (nullable = true)
 |-- 1/27/20: string (nullable = true)
 |-- 1/28/20: string (nullable = true)
 |-- 1/29/20: string (nullable = true)
 |-- 1/30/20: string (nullable = true)
 |-- 1/31/20: string (nullable = true)
 |-- 2/1/20: string (nullable = true)
 |-- 2/2/20: string (nullable = true)
 |-- 2/3/20: string (nullable = true)
 |-- 2/4/20: string (nullable = true)
 |-- 2/5/20: string (nullable = true)
 |-- 2/6/20: string (nullable = true)
 |-- 2/7/20: string (nullable = true)
 |-- 2/8/20: string (nullable = true)
 |-- 2/9/20: string (nullable = true)
 |-- 2/10/20: string (nullable = true)
 |-- 2/11/20: string (nullable = true)
 |-- 2/12/20: string (

### Your Task

We are aware of the great deal of stress we are all experiencing because of the spread of the Coronavirus. As such, we decided to conclude our series of Colabs with a **lightweight task** -- given everything you have learned about Spark during the quarter, this Colab should take you just a few minutes to complete.

At the same time, we turned this into an opportunity to raise awareness about the extent of the COVID-19 epidemic.

[Stay healthy, wash your hands often](https://www.cdc.gov/coronavirus/2019-ncov/about/index.html), and invest the time you saved on this Colab to be emotionally supportive to your friends and family!

Signed, *the CS246 teaching staff*

In [13]:
def md5(s):
  import hashlib
  m = hashlib.md5()
  m.update(s.encode())
  return m.hexdigest()[:2]

In [16]:
md5('3/1/20')

'e7'

Consider only the most recent data points in the timeseries, and compute:


*   number of confirmed COVID-19 cases across the globe
*   number of deaths due to COVID-19 (across the globe)
*   number of recovered patients across the globe



In [17]:
confirmed.show()


+--------------+--------------+------------------+------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+
|Province/State|Country/Region|               Lat|              Long|1/22/20|1/23/20|1/24/20|1/25/20|1/26/20|1/27/20|1/28/20|1/29/20|1/30/20|1/31/20|2/1/20|2/2/20|2/3/20|2/4/20|2/5/20|2/6/20|2/7/20|2/8/20|2/9/20|2/10/20|2/11/20|2/12/20|2/13/20|2/14/20|2/15/20|2/16/20|2/17/20|2/18/20|2/19/20|2/20/20|2/21/20|2/22/20|2/23/20|2/24/20|2/25/20|2/26/20|2/27/20|2/28/20|2/29/20|3/1/20|3/2/20|3/3/20|3/4/20|3/5/20|
+--------------+--------------+------------------+------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+

In [33]:
# Flatmap the number of confirmed cases in different places
confirmed_flatten = confirmed.select('3/5/20').rdd.flatMap(lambda x: x).collect()

# Flattenmap the number of death cases in different places
deaths_flatten = deaths.select('3/5/20').rdd.flatMap(lambda x: x).collect()

# Flattenmap the number of recovered cases in different places
recovered_flatten = recovered.select('3/5/20').rdd.flatMap(lambda x: x).collect()

In [37]:
def count_total(lst):
  # Initialize the total number
  tot = 0
  # Iterate through all the locations
  for num_case in lst:
    tot += int(num_case)
  return tot

In [39]:
print("Total num of confirmed cases in the world: {}".format(count_total(confirmed_flatten)))
print("Total num of death cases in the world: {}".format(count_total(deaths_flatten)))
print("Total num of recovered cases in the world: {}".format(count_total(recovered_flatten)))


Total num of confirmed cases in the world: 97886
Total num of death cases in the world: 3348
Total num of recovered cases in the world: 53797


Consider only the most recent data points in the timeseries, and filter out the geographical locations where less than 50 cases have been confirmed.
For the areas still taken into consideration after the filtering step, compute the ratio between number of recovered patients and number of confirmed cases.

In [41]:
confirmed.show()

+--------------+--------------+------------------+------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+
|Province/State|Country/Region|               Lat|              Long|1/22/20|1/23/20|1/24/20|1/25/20|1/26/20|1/27/20|1/28/20|1/29/20|1/30/20|1/31/20|2/1/20|2/2/20|2/3/20|2/4/20|2/5/20|2/6/20|2/7/20|2/8/20|2/9/20|2/10/20|2/11/20|2/12/20|2/13/20|2/14/20|2/15/20|2/16/20|2/17/20|2/18/20|2/19/20|2/20/20|2/21/20|2/22/20|2/23/20|2/24/20|2/25/20|2/26/20|2/27/20|2/28/20|2/29/20|3/1/20|3/2/20|3/3/20|3/4/20|3/5/20|
+--------------+--------------+------------------+------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+

In [75]:
# Filter only the date and the locations
confirmed_cases = confirmed.select('Province/State', 'Country/Region', col('3/5/20').alias("confirmed_cases"))
death_cases =deaths.select('Province/State', 'Country/Region', col('3/5/20').alias('death_cases'))
recovered_cases = recovered.select('Province/State', 'Country/Region', col('3/5/20').alias('recovered_cases'))

# Join the two lists
joined = confirmed_cases.join(death_cases, ['Province/State', 'Country/Region'])
joined = joined.join(recovered_cases, ['Province/State', 'Country/Region'])

# Create a column called "percent_recovered(%)" to compute the percentage of recovered 
result = joined.withColumn("percent_recovered(%)", f.round(joined.recovered_cases / joined.confirmed_cases * 100, 2))

# Create a column called "percent_deaths(%)", to compute the percentage of deaths
result = result.withColumn("percent_deaths(%)", f.round(result.death_cases / result.confirmed_cases * 100, 2))



In [81]:
# Show the result where the total confirmed cases is less than 50, and sort then in a descending order
sort_recovered = result.filter(result.confirmed_cases < 50).sort("percent_recovered(%)", ascending = False).show()


+--------------------+--------------+---------------+-----------+---------------+--------------------+-----------------+
|      Province/State|Country/Region|confirmed_cases|death_cases|recovered_cases|percent_recovered(%)|percent_deaths(%)|
+--------------------+--------------+---------------+-----------+---------------+--------------------+-----------------+
|         Madison, WI|            US|              1|          0|              1|               100.0|              0.0|
|             Qinghai|Mainland China|             18|          0|             18|               100.0|              0.0|
|           Tempe, AZ|            US|              1|          0|              1|               100.0|              0.0|
|          London, ON|        Canada|              1|          0|              1|               100.0|              0.0|
|               Tibet|Mainland China|              1|          0|              1|               100.0|              0.0|
|          Boston, MA|          

In [82]:
sort_deaths = result.filter(result.confirmed_cases < 50).sort("percent_deaths(%)", ascending = False).show()


+--------------------+--------------+---------------+-----------+---------------+--------------------+-----------------+
|      Province/State|Country/Region|confirmed_cases|death_cases|recovered_cases|percent_recovered(%)|percent_deaths(%)|
+--------------------+--------------+---------------+-----------+---------------+--------------------+-----------------+
|   Placer County, CA|            US|              2|          1|              0|                 0.0|             50.0|
|   Western Australia|     Australia|              3|          1|              0|                 0.0|            33.33|
|Snohomish County, WA|            US|             18|          1|              0|                 0.0|             5.56|
|     New South Wales|     Australia|             22|          1|              4|               18.18|             4.55|
|              Taiwan|        Taiwan|             44|          1|             12|               27.27|             2.27|
|Unassigned Locati...|          